In [25]:
import import_hack
import steps
from data_store import sqlite_table_datastore as sqlds
from data_store import file_system_directory_datastore as fsds
from data_store import stream_ndarray_adapter_datastore as snads
from data_store import numpy_datastore as npds
from transformer import transformers as trs
from search import inverted_multi_index_searcher as imis
from search import exhaustive_searcher as exs
from quantization import pq_quantizer
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
img_dir_path = r'C:\data\images\brodatz\data.brodatz\size_213x213'
sqlite_db_path='ds_data\example_find_nearest_by_histograms_with_multi_index'
sqlite_table_name='histograms'
n_subquantizers=2
n_clusters=256
sqltie_clusters_table_name='clusters_pq_{0}_{1}'.format(n_subquantizers, n_clusters)

In [3]:
image_bytes_ds = fsds.FileSystemDirectoryDataStore(dir_path=img_dir_path)
histograms_ds=sqlds.SQLiteTableDataStore(sqlite_db_path, sqlite_table_name)
image_bytes_ds.get_count()

999

In [4]:
transformers_=[trs.BytesToNdarray(), trs.NdarrayToOpencvMatrix(), trs.OpencvMatrixToHistogram(graycolor=True)]
steps.transform_step(image_bytes_ds, transformers_, histograms_ds)

In [5]:
histograms_ds_ndarrayds = snads.StreamNdarrayAdapterDataStore(histograms_ds)
print(histograms_ds_ndarrayds.get_items_sorted_by_ids().shape)
np_neighbor_ids_ds=npds.NumpyDataStore()

(999, 256, 1)


In [8]:
x=histograms_ds_ndarrayds.get_items_sorted_by_ids()
x_ids=histograms_ds_ndarrayds.get_ids_sorted()

In [10]:
pq_=pq_quantizer.PQQuantizer(n_clusters, n_subquantizers)
#be careful with pq: multiindex will consume at least (n_clusters^m_subquantizers)*4 bytes
cluster_centers_ds=sqlds.SQLiteTableDataStore(sqlite_db_path, sqltie_clusters_table_name)
steps.quantize_step(histograms_ds_ndarrayds, pq_, cluster_centers_ds)

In [11]:
cluster_centers_ndarray_ds=snads.StreamNdarrayAdapterDataStore(cluster_centers_ds)
cluster_centers=cluster_centers_ndarray_ds.get_items_sorted_by_ids()

In [12]:
cluster_centers.shape

(2, 256, 128)

In [56]:
imi_searcher=imis.InvertedMultiIndexSearcher(x, x_ids, cluster_centers)
exs_searcher=exs.ExhaustiveSearcher(x, x_ids)

In [57]:
%%timeit -n10 -r10
imi_searcher.find_nearest_ids(x[0].reshape((1,256)), 5)

10 loops, best of 10: 143 µs per loop


In [60]:
%%timeit -n10 -r10
exs_searcher.find_nearest_ids(x[0].reshape((1,256)), 5)

10 loops, best of 10: 1.84 ms per loop


C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\skl